# Benchmarking the non-deep descriptors

Here we will evaluate all descriptors, which are available easily enough, e.g. from OpenCV or other python implementation. There will be final comparison table in the end of this notebook.


### Disclaimer: binary descriptors are not typically run on normalized patches


E.g. this benchmark shows quite strong performance of BRIEF descriptor as implemented in skimage. However,
typically binary descriptors are optimized for speed and run on the (non-normalized) image, rather than cropped-blurred-rescaled patches. That is why BRIEF-in-the-wild or FREAK-in-the-wild would probably perform noticably worse than, currect benchmark shows

In [1]:
#!pip install opencv-python --user
#!pip install opencv-contrib-python --user 
#!pip install kornia

# SIFT and RootSIFT, [kornia](https://kornia.readthedocs.io/en/latest/feature.html#kornia.feature.SIFTDescriptor) implementation

In [2]:
from IPython.display import clear_output
import torch
import kornia
from brown_phototour_revisited.benchmarking import *

descs_out_dir = 'data/descriptors'
download_dataset_to = 'data/dataset'
results_dir = 'data/mAP'

full_results_dict = {}

for patch_size in [32, 41]:
    desc_name = 'Kornia RootSIFT'
    model = kornia.feature.SIFTDescriptor(patch_size, rootsift=True).eval()
    desc_dict = full_evaluation(model,
                                desc_name,
                                path_to_save_dataset = download_dataset_to,
                                path_to_save_descriptors = descs_out_dir,
                                path_to_save_mAP = results_dir,
                                patch_size = patch_size, 
                                device = torch.device('cuda:0'), 
                           distance='euclidean',
                           backend='pytorch-cuda')
    full_results_dict[f'{desc_name} {patch_size}px'] = desc_dict
clear_output()
print_results_table(full_results_dict)

------------------------------------------------------------------------------
Mean Average Precision wrt Lowe SNN ratio criterion on UBC Phototour Revisited
------------------------------------------------------------------------------
trained on       liberty notredame  liberty yosemite  notredame yosemite
tested  on           yosemite           notredame            liberty
------------------------------------------------------------------------------
Kornia RootSIFT 32px   58.24              49.07               49.65 
Kornia RootSIFT 41px   57.83              48.48               49.01 
------------------------------------------------------------------------------


In [3]:
for patch_size in [32, 41]:
    desc_name = 'Kornia SIFT'
    model = kornia.feature.SIFTDescriptor(patch_size, rootsift=False).eval()
    desc_dict = full_evaluation(model,
                                'Kornia_SIFT',
                                path_to_save_dataset = download_dataset_to,
                                path_to_save_descriptors = descs_out_dir,
                                path_to_save_mAP = results_dir,
                                patch_size = patch_size, 
                                device = torch.device('cuda:0'), 
                           distance='euclidean',
                           backend='pytorch-cuda')
    full_results_dict[f'{desc_name} {patch_size}px'] = desc_dict
clear_output()
print_results_table(full_results_dict)

------------------------------------------------------------------------------
Mean Average Precision wrt Lowe SNN ratio criterion on UBC Phototour Revisited
------------------------------------------------------------------------------
trained on       liberty notredame  liberty yosemite  notredame yosemite
tested  on           yosemite           notredame            liberty
------------------------------------------------------------------------------
Kornia RootSIFT 32px   58.24              49.07               49.65 
Kornia RootSIFT 41px   57.83              48.48               49.01 
Kornia SIFT 32px       58.47              47.76               48.70 
Kornia SIFT 41px       58.14              47.30               48.30 
------------------------------------------------------------------------------


# SIFT OpenCV

Now we will try OpenCV SIFT. It expects not patches, but full image and keypoints, so we will create a single keypoint in the center of the patch.

In [4]:
# copy-adopted from here https://github.com/hpatches/hpatches-benchmark/blob/master/python/extract_opencv_sift.py
import cv2
import numpy as np


def get_center_kp(PS):
    c = PS/2.0
    center_kp = cv2.KeyPoint()
    center_kp.pt = (float(c),float(c))
    center_kp.size = PS/5.303
    return center_kp

sift = cv2.SIFT_create()

descs_out_dir = 'data/descriptors'
download_dataset_to = 'data/dataset'


for PS in [32, 41]:
    desc_name = 'OpenCV_SIFT'
    def extract_opencv_sift(patch):
        return sift.compute((255*patch).astype(np.uint8),[get_center_kp(PS)])[1][0].reshape(128)
    desc_dict = full_evaluation(extract_opencv_sift,
                                desc_name,
                                path_to_save_dataset = download_dataset_to,
                                path_to_save_descriptors = descs_out_dir,
                                path_to_save_mAP = results_dir,
                                patch_size = PS, 
                                device = torch.device('cuda:0'), 
                           distance='euclidean',
                           backend='pytorch-cuda')
    full_results_dict[f'{desc_name} {PS}px'] = desc_dict
clear_output()
print_results_table(full_results_dict)


------------------------------------------------------------------------------
Mean Average Precision wrt Lowe SNN ratio criterion on UBC Phototour Revisited
------------------------------------------------------------------------------
trained on       liberty notredame  liberty yosemite  notredame yosemite
tested  on           yosemite           notredame            liberty
------------------------------------------------------------------------------
Kornia RootSIFT 32px   58.24              49.07               49.65 
Kornia RootSIFT 41px   57.83              48.48               49.01 
Kornia SIFT 32px       58.47              47.76               48.70 
Kornia SIFT 41px       58.14              47.30               48.30 
OpenCV_SIFT 32px       53.16              45.93               46.00 
OpenCV_SIFT 41px       54.10              46.09               46.29 
------------------------------------------------------------------------------


## OpenCV RootSIFT

In [5]:
import cv2
import numpy as np

sift = cv2.SIFT_create()
def extract_opencv_rootsift(patch):
    desc =  sift.compute((255*patch).astype(np.uint8),[get_center_kp(PS)])[1][0].reshape(1,-1)
    desc /= desc.sum(axis=1, keepdims=True) + 1e-8
    desc = np.sqrt(desc)
    return desc


for PS in [32, 41]:
    desc_name = 'OpenCV_RootSIFT'
    desc_dict = full_evaluation(extract_opencv_rootsift,
                                desc_name,
                                path_to_save_dataset = download_dataset_to,
                                path_to_save_descriptors = descs_out_dir,
                                path_to_save_mAP = results_dir,
                                patch_size = PS, 
                                device = torch.device('cuda:0'), 
                           distance='euclidean',
                           backend='pytorch-cuda')
    full_results_dict[f'{desc_name} {PS}px'] = desc_dict
clear_output()
print_results_table(full_results_dict)

------------------------------------------------------------------------------
Mean Average Precision wrt Lowe SNN ratio criterion on UBC Phototour Revisited
------------------------------------------------------------------------------
trained on       liberty notredame  liberty yosemite  notredame yosemite
tested  on           yosemite           notredame            liberty
------------------------------------------------------------------------------
Kornia RootSIFT 32px   58.24              49.07               49.65 
Kornia RootSIFT 41px   57.83              48.48               49.01 
Kornia SIFT 32px       58.47              47.76               48.70 
Kornia SIFT 41px       58.14              47.30               48.30 
OpenCV_SIFT 32px       53.16              45.93               46.00 
OpenCV_SIFT 41px       54.10              46.09               46.29 
OpenCV_RootSIFT 32px   53.50              47.16               47.37 
OpenCV_RootSIFT 41px   54.19              47.20            

What else do we have in OpenCV 4.4? 
- VGG (ConvexOpt). It is very cool descriptor, but there is no information about which subset provided descriptors were learned on :(
- LATCH (learned on yosemite)
- BoostDesc. Same as VGG
- DAISY. Same as VGG
- FREAK
- LUCID
- some others, which I haven't yet tried.

Let's go!

# [LATCH](https://arxiv.org/abs/1501.03719)

It is binary learned descriptor, learned on `yosemite` sequence, as stated in the paper.

In [6]:
desc = cv2.xfeatures2d.LATCH_create()

def extract_latch(patch):
    return desc.compute((255*patch).astype(np.uint8),
                        [get_center_kp(PS)])[1][0].reshape(-1)

descs_out_dir = 'data/descriptors'
download_dataset_to = 'data/dataset'

for PS in [65]: #32  crashes for LATCH.
    desc_name = 'OpenCV_LATCH'
    models = {"yosemite": extract_latch}
    desc_dict = full_evaluation(models,
                                desc_name,
                                path_to_save_dataset = download_dataset_to,
                                path_to_save_descriptors = descs_out_dir,
                                path_to_save_mAP = results_dir,
                                patch_size = PS, 
                                device = torch.device('cuda:0'), 
                           distance='hamming',
                           backend='numpy')
    full_results_dict[f'{desc_name} {PS}px'] = desc_dict
clear_output()
print_results_table(full_results_dict)

------------------------------------------------------------------------------
Mean Average Precision wrt Lowe SNN ratio criterion on UBC Phototour Revisited
------------------------------------------------------------------------------
trained on       liberty notredame  liberty yosemite  notredame yosemite
tested  on           yosemite           notredame            liberty
------------------------------------------------------------------------------
Kornia RootSIFT 32px   58.24              49.07               49.65 
Kornia RootSIFT 41px   57.83              48.48               49.01 
Kornia SIFT 32px       58.47              47.76               48.70 
Kornia SIFT 41px       58.14              47.30               48.30 
OpenCV_SIFT 32px       53.16              45.93               46.00 
OpenCV_SIFT 41px       54.10              46.09               46.29 
OpenCV_RootSIFT 32px   53.50              47.16               47.37 
OpenCV_RootSIFT 41px   54.19              47.20            

# [LUCID](https://papers.nips.cc/paper/4706-locally-uniform-comparison-image-descriptor)

That is very simple descriptor - sorted pxiel intensities. 

In [7]:
desc = cv2.xfeatures2d.LUCID_create()
def extract_lucid(patch):
    return desc.compute(cv2.cvtColor((255*patch).astype(np.uint8),cv2.COLOR_GRAY2RGB),
                        [get_center_kp(PS)])[1][0].reshape(-1)

descs_out_dir = 'data/descriptors'
download_dataset_to = 'data/dataset'

for PS in [32]:
    desc_name = 'OpenCV_LUCID'
    desc_dict = full_evaluation(extract_lucid,
                                desc_name,
                                path_to_save_dataset = download_dataset_to,
                                path_to_save_descriptors = descs_out_dir,
                                path_to_save_mAP = results_dir,
                                patch_size = PS, 
                                device = torch.device('cuda:0'), 
                           distance='hamming',
                           backend='numpy')
    full_results_dict[f'{desc_name} {PS}px'] = desc_dict
clear_output()
print_results_table(full_results_dict)

------------------------------------------------------------------------------
Mean Average Precision wrt Lowe SNN ratio criterion on UBC Phototour Revisited
------------------------------------------------------------------------------
trained on       liberty notredame  liberty yosemite  notredame yosemite
tested  on           yosemite           notredame            liberty
------------------------------------------------------------------------------
Kornia RootSIFT 32px   58.24              49.07               49.65 
Kornia RootSIFT 41px   57.83              48.48               49.01 
Kornia SIFT 32px       58.47              47.76               48.70 
Kornia SIFT 41px       58.14              47.30               48.30 
OpenCV_SIFT 32px       53.16              45.93               46.00 
OpenCV_SIFT 41px       54.10              46.09               46.29 
OpenCV_RootSIFT 32px   53.50              47.16               47.37 
OpenCV_RootSIFT 41px   54.19              47.20            

In [8]:
import numpy as np
from skimage.feature import BRIEF
def extract_skimage_BRIEF(patch):
    BR.extract(patch.astype(np.float64), np.array([PS/2.0, PS/2.0]).reshape(1,2))
    return BR.descriptors.astype(np.float32)

descs_out_dir = 'data/descriptors'
download_dataset_to = 'data/dataset'

for PS in [65]:
    BR = BRIEF(patch_size = PS)
    desc_name = 'skimage_BRIEF'
    desc_dict = full_evaluation(extract_skimage_BRIEF,
                                desc_name,
                                path_to_save_dataset = download_dataset_to,
                                path_to_save_descriptors = descs_out_dir,
                                path_to_save_mAP = results_dir,
                                patch_size = PS, 
                                device = torch.device('cuda:0'), 
                           distance='hamming',
                           backend='numpy')
    full_results_dict[f'{desc_name} {PS}px'] = desc_dict
clear_output()
print_results_table(full_results_dict)

------------------------------------------------------------------------------
Mean Average Precision wrt Lowe SNN ratio criterion on UBC Phototour Revisited
------------------------------------------------------------------------------
trained on       liberty notredame  liberty yosemite  notredame yosemite
tested  on           yosemite           notredame            liberty
------------------------------------------------------------------------------
Kornia RootSIFT 32px   58.24              49.07               49.65 
Kornia RootSIFT 41px   57.83              48.48               49.01 
Kornia SIFT 32px       58.47              47.76               48.70 
Kornia SIFT 41px       58.14              47.30               48.30 
OpenCV_SIFT 32px       53.16              45.93               46.00 
OpenCV_SIFT 41px       54.10              46.09               46.29 
OpenCV_RootSIFT 32px   53.50              47.16               47.37 
OpenCV_RootSIFT 41px   54.19              47.20            

# RootSIFTPCA

Now, let's try to learn [PCA-RootSIFT](https://hal.inria.fr/hal-01145656/document).  We will need to first extract all patches, learn PCA and evaluate it normally - train on 1 dataset, test on 2 other.

In [9]:
from brown_phototour_revisited.extraction import extract_pytorchinput_descriptors
from sklearn.decomposition import PCA

patch_size = 32
desc_name = 'Kornia RootSIFT'
descs_out_dir = 'data/descriptors'
download_dataset_to = 'data/dataset'
model = kornia.feature.SIFTDescriptor(patch_size, rootsift=True).eval()
descriptors = {}
PCAs = {}
for ds_name in ['liberty', 'notredame', 'yosemite']:
    descriptors[ds_name] = extract_pytorchinput_descriptors(model,
                                                            desc_name,
                                                            ds_name,
                                                            download_dataset_to,
                                                            descs_out_dir,
                                                            patch_size,
                                                            device = torch.device('cuda:0'))['descriptors']
    
    PCAs[ds_name] = PCA(128)
    PCAs[ds_name].fit(descriptors[ds_name])

# Found cached data data/dataset/liberty.pt
data/descriptors/Kornia RootSIFT_32px_liberty.npy already exists, loading
# Found cached data data/dataset/notredame.pt
data/descriptors/Kornia RootSIFT_32px_notredame.npy already exists, loading
# Found cached data data/dataset/yosemite.pt
data/descriptors/Kornia RootSIFT_32px_yosemite.npy already exists, loading


Now lets convert PCA into pytorch model and apply root normalization then

In [10]:
from torch import nn
import torch.nn.functional as F
class RootSIFTPCA(nn.Module):
    def __init__(self, PS, PCA):
        super(RootSIFTPCA, self).__init__()
        self.rootsift = kornia.feature.SIFTDescriptor(PS, rootsift=True)
        self.register_buffer('mean', torch.from_numpy(PCA.mean_).float())
        self.register_buffer('components', torch.from_numpy(PCA.components_).float())
        return
    def forward(self, x):
        rootsift = self.rootsift(x)
        pca = torch.mm(rootsift - self.mean, self.components.t())
        pca_root = torch.sqrt(torch.abs(pca)) * torch.sign(pca)
        return F.normalize(pca_root, dim=1, p=2)

In [11]:
models = {}
patch_size = 32
for ds_name in ['liberty', 'notredame', 'yosemite']:
    models[ds_name] = RootSIFTPCA(patch_size, PCAs[ds_name])
    

desc_name = 'Kornia RootSIFTPCA'
desc_dict = full_evaluation(models,
                            desc_name,
                            path_to_save_dataset = download_dataset_to,
                            path_to_save_descriptors = descs_out_dir,
                            path_to_save_mAP = results_dir,
                            patch_size = patch_size, 
                            device = torch.device('cuda:0'), 
                       distance='euclidean',
                       backend='pytorch-cuda')
full_results_dict[f'{desc_name} {patch_size}px'] = desc_dict
clear_output()
print_results_table(full_results_dict)



------------------------------------------------------------------------------
Mean Average Precision wrt Lowe SNN ratio criterion on UBC Phototour Revisited
------------------------------------------------------------------------------
trained on       liberty notredame  liberty yosemite  notredame yosemite
tested  on           yosemite           notredame            liberty
------------------------------------------------------------------------------
Kornia RootSIFT 32px   58.24              49.07               49.65 
Kornia RootSIFT 41px   57.83              48.48               49.01 
Kornia SIFT 32px       58.47              47.76               48.70 
Kornia SIFT 41px       58.14              47.30               48.30 
OpenCV_SIFT 32px       53.16              45.93               46.00 
OpenCV_SIFT 41px       54.10              46.09               46.29 
OpenCV_RootSIFT 32px   53.50              47.16               47.37 
OpenCV_RootSIFT 41px   54.19              47.20            


## Multiple Kernel Local Descriptor


Now let's check the most advanced hand-crafted and learned, but not deep descriptor - MKD.
It is available from https://github.com/manyids2/mkd_local_descriptor and described in [Understanding and Improving Kernel Local Descriptors](https://arxiv.org/abs/1811.11147) IJCV 2018 paper.

In [12]:
#!pip install https://github.com/manyids2/mkd_local_descriptor/archive/1.0.2.tar.gz --user

In [13]:
import mkd_local_descriptor as mm
# Use model_file trained on PhotoTourism for 32x32 patch_size.
patch_size = 32
for white in [None, 'pca', 'pcaws', 'pcawt', 'lw']:
    models = {}
    for ds_name in ['liberty', 'notredame', 'yosemite']:
        models[ds_name] =  mm.MKD(dtype='concat',             # 'concat', 'polar', 'cart'.
                 patch_size=patch_size,
                 whitening=white,             # None, 'lw', 'pca', 'pcaws', 'pcawt'.
                 training_set=ds_name,     # 'liberty', 'notredame', 'yosemite'
                 reduce_dims=128,
                 do_l2=True,
                 do_final_l2=True,
                 do_gmask=True,
                 device='cuda')
    desc_name = f'MKD-concat-{white}-{patch_size}'
    desc_dict = full_evaluation(models,
                                desc_name,
                                path_to_save_dataset = download_dataset_to,
                                path_to_save_descriptors = descs_out_dir,
                                path_to_save_mAP = results_dir,
                                patch_size = patch_size, 
                                device = torch.device('cuda:0'), 
                           distance='euclidean',
                           backend='pytorch-cuda')
    full_results_dict[f'{desc_name} {patch_size}px'] = desc_dict
    clear_output()
    print_results_table(full_results_dict)

------------------------------------------------------------------------------
Mean Average Precision wrt Lowe SNN ratio criterion on UBC Phototour Revisited
------------------------------------------------------------------------------
trained on       liberty notredame  liberty yosemite  notredame yosemite
tested  on           yosemite           notredame            liberty
------------------------------------------------------------------------------
Kornia RootSIFT 32px   58.24              49.07               49.65 
Kornia RootSIFT 41px   57.83              48.48               49.01 
Kornia SIFT 32px       58.47              47.76               48.70 
Kornia SIFT 41px       58.14              47.30               48.30 
OpenCV_SIFT 32px       53.16              45.93               46.00 
OpenCV_SIFT 41px       54.10              46.09               46.29 
OpenCV_RootSIFT 32px   53.50              47.16               47.37 
OpenCV_RootSIFT 41px   54.19              47.20            

Wow, MKD-concat-lw-32 seems to be very good! Let's compare it to HardNet

In [14]:
desc_name = 'HardNet'
patch_size = 32
desc_dict =  load_cached_results(desc_name,
                    learned_on = ['liberty', 'notredame', 'yosemite'],
                    path_to_save_dataset = download_dataset_to,
                    path_to_save_descriptors = descs_out_dir,
                    path_to_save_mAP = results_dir,
                    patch_size = patch_size)

full_results_dict[f'{desc_name} {patch_size}px'] = desc_dict
clear_output()
print_results_table(full_results_dict)

------------------------------------------------------------------------------
Mean Average Precision wrt Lowe SNN ratio criterion on UBC Phototour Revisited
------------------------------------------------------------------------------
trained on       liberty notredame  liberty yosemite  notredame yosemite
tested  on           yosemite           notredame            liberty
------------------------------------------------------------------------------
Kornia RootSIFT 32px   58.24              49.07               49.65 
Kornia RootSIFT 41px   57.83              48.48               49.01 
Kornia SIFT 32px       58.47              47.76               48.70 
Kornia SIFT 41px       58.14              47.30               48.30 
OpenCV_SIFT 32px       53.16              45.93               46.00 
OpenCV_SIFT 41px       54.10              46.09               46.29 
OpenCV_RootSIFT 32px   53.50              47.16               47.37 
OpenCV_RootSIFT 41px   54.19              47.20            

If you use the benchmark, please cite it:

    @misc{BrownRevisited2020,
      title={UBC PhotoTour Revisied},
      author={Mishkin, Dmytro},
      year={2020},
      url = {https://github.com/ducha-aiki/brown_phototour_revisited}
    }